## Bibliotecas 

In [2]:
import pandas as pd
import duckdb
from pathlib import Path
import os
import requests
import warnings
warnings.filterwarnings('ignore')

print(" Notebook: Organizar Dados Brutos")
print(" Objetivo: Carregar SIM + Atlas Brasil + IBGE no DuckDB")
print("="*60)


 Notebook: Organizar Dados Brutos
 Objetivo: Carregar SIM + Atlas Brasil + IBGE no DuckDB


In [3]:
BASE_DIR = Path("..")
DATA_DIR = BASE_DIR / "data"
DATA_RAW = DATA_DIR / "raw"
DATA_RAW_SIM = DATA_RAW / "sim"
DATA_RAW_ATLAS = DATA_RAW / "atlas"
DATA_PROCESSED = DATA_DIR / "processed"
DATA_FINAL = DATA_DIR / "final"
DUCKDB_DIR = BASE_DIR / "duckdb"

# Criar diretórios se não existirem
DATA_PROCESSED.mkdir(parents=True, exist_ok=True)
DATA_FINAL.mkdir(parents=True, exist_ok=True)

print(" Estrutura de pastas verificada!")
print(f"    {DATA_RAW_SIM}")
print(f"    {DATA_RAW_ATLAS}")
print(f"    {DATA_PROCESSED}")
print(f"    {DATA_FINAL}")
print(f"    {DUCKDB_DIR}")

 Estrutura de pastas verificada!
    ../data/raw/sim
    ../data/raw/atlas
    ../data/processed
    ../data/final
    ../duckdb


conectando ao dunkDB

In [4]:
DB_PATH = DUCKDB_DIR / "mortalidade.duckdb"
conn = duckdb.connect(str(DB_PATH))

print(f" Conexão DuckDB estabelecida!")
print(f" Banco: {DB_PATH}")

# Verificar versão
version = conn.execute("SELECT version()").fetchone()[0]
print(f" Versão: {version}")

 Conexão DuckDB estabelecida!
 Banco: ../duckdb/mortalidade.duckdb
 Versão: v1.4.1


## carregar Atlas brasil 

In [4]:
print("\n" + "="*60)
print(" CARREGANDO ATLAS BRASIL")
print("="*60)

# Procurar arquivo do Atlas
atlas_file = None
for pattern in ['data.xlsx', 'atlas*.xlsx', 'atlas*.csv']:
    files = list(DATA_RAW_ATLAS.glob(pattern))
    if files:
        atlas_file = files[0]
        break

if atlas_file:
    print(f"\n Arquivo encontrado: {atlas_file.name}")
    
    try:
        # Carregar Excel
        df_atlas = pd.read_excel(atlas_file)
        
        print(f"✅ Carregado com sucesso!")
        print(f"📊 Shape: {df_atlas.shape}")
        print(f"\n📋 Colunas originais ({len(df_atlas.columns)}):")
        for i, col in enumerate(df_atlas.columns, 1):
            print(f"   {i:2d}. {col}")
        
    except Exception as e:
        print(f" Erro ao carregar: {e}")
        df_atlas = None
else:
    print(" Arquivo do Atlas Brasil não encontrado!")
    df_atlas = None


 CARREGANDO ATLAS BRASIL

 Arquivo encontrado: data.xlsx
✅ Carregado com sucesso!
📊 Shape: (5569, 20)

📋 Colunas originais (20):
    1. Territorialidades
    2. Razão 10% mais ricos / 40% mais pobres 2010
    3. Índice de Gini 2010
    4. % de 15 a 24 anos de idade que não estudam nem trabalham em domicílios vulneráveis à pobreza 2010
    5. % de pessoas em domicílios sem energia elétrica 2010
    6. % de pessoas em domicílios com abastecimento de água e esgotamento sanitário inadequados 2010
    7. População total 2010
    8. Mortalidade infantil 2010
    9. Razão de dependência 2010
   10. Taxa de envelhecimento 2010
   11. Taxa de analfabetismo - 25 anos ou mais de idade 2010
   12. Taxa de analfabetismo - 15 anos ou mais de idade 2010
   13. % de 18 anos ou mais de idade com ensino fundamental completo 2010
   14. % de 25 anos ou mais de idade com ensino médio completo 2010
   15. % de 25 anos ou mais de idade com ensino superior completo 2010
   16. Expectativa de anos de estudo 

## Limpeza e padronização de colunas do Atlas 

In [5]:
if df_atlas is not None:
    print("\n" + "="*60)
    print("🧹 LIMPANDO E PADRONIZANDO COLUNAS")
    print("="*60)
    
    # Mapear nomes das colunas para formato limpo
    col_mapping = {}
    
    for col in df_atlas.columns:
        # Remover " 2010" do final
        new_col = col.replace(' 2010', '')
        
        # Substituir caracteres especiais
        new_col = (new_col
                   .replace('% de ', 'perc_')
                   .replace('%', 'perc')
                   .replace(' / ', '_')
                   .replace('/', '_')
                   .replace(' - ', '_')
                   .replace(' ou mais', '+')
                   .replace(' de idade', '')
                   .replace(' anos', 'a')
                   .replace(' ', '_')
                   .lower())
        
        col_mapping[col] = new_col
    
    # Renomear colunas
    df_atlas_clean = df_atlas.rename(columns=col_mapping)
    
    print(f"\n Colunas padronizadas!")
    print(f"\n Novos nomes:")
    for old, new in list(col_mapping.items())[:10]:
        print(f"   {old[:40]:40s} → {new}")
    if len(col_mapping) > 10:
        print(f"   ... e mais {len(col_mapping)-10} colunas")



🧹 LIMPANDO E PADRONIZANDO COLUNAS

 Colunas padronizadas!

 Novos nomes:
   Territorialidades                        → territorialidades
   Razão 10% mais ricos / 40% mais pobres 2 → razão_10perc_mais_ricos_40perc_mais_pobres
   Índice de Gini 2010                      → índice_de_gini
   % de 15 a 24 anos de idade que não estud → perc_15_a_24a_que_não_estudam_nem_trabalham_em_domicílios_vulneráveis_à_pobreza
   % de pessoas em domicílios sem energia e → perc_pessoas_em_domicílios_sem_energia_elétrica
   % de pessoas em domicílios com abastecim → perc_pessoas_em_domicílios_com_abastecimento_de_água_e_esgotamento_sanitário_inadequados
   População total 2010                     → população_total
   Mortalidade infantil 2010                → mortalidade_infantil
   Razão de dependência 2010                → razão_de_dependência
   Taxa de envelhecimento 2010              → taxa_de_envelhecimento
   ... e mais 10 colunas


## Extrair codigo de Municipios 

In [ ]:
if df_atlas is not None:
    print("\n" + "="*60)
    print(" EXTRAINDO CÓDIGO DO MUNICÍPIO")
    print("="*60)
    
    # A coluna "Territorialidades" contem: "Nome do Municipio (UF)"
    # temos que ter o codigo do municipio no IBGE
    
    print("\n Baixando códigos de municípios do IBGE...")
    
    try:
        url = "https://servicodados.ibge.gov.br/api/v1/localidades/municipios"
        response = requests.get(url, timeout=30)
        municipios_ibge = response.json()
        
        # Criar lista de dados do IBGE
        data_ibge = []

        for m in municipios_ibge:
            try:
                uf_sigla = m.get('microrregiao', {}).get('mesorregiao', {}).get('UF', {}).get('sigla')
                if uf_sigla:
                    data_ibge.append({
                        'municipio_uf': f"{m['nome']} ({uf_sigla})",
                        'cod_municipio': str(m['id'])[:6],
                        'municipio': m['nome'],
                        'uf': uf_sigla
                    })
            except Exception:
                # Ignora casos raros de estrutura irregular
                continue

        # Converter em DataFrame
        df_codigos = pd.DataFrame(data_ibge)
        print(f" {len(df_codigos)} códigos obtidos do IBGE")

        # Fazer merge com Atlas
        df_atlas_clean = df_atlas_clean.merge(
            df_codigos,
            left_on='territorialidades',
            right_on='municipio_uf',
            how='left'
        )

        # Verificar merge
        matched = df_atlas_clean['cod_municipio'].notna().sum()
        total = len(df_atlas_clean)

        print(f" Merge concluído: {matched}/{total} municípios com código")

        if matched < total:
            print(f"  {total - matched} municípios sem código (verificar depois)")

    except Exception as e:
        print(f" Erro ao baixar códigos IBGE: {e}")



 EXTRAINDO CÓDIGO DO MUNICÍPIO

 Baixando códigos de municípios do IBGE...
 5570 códigos obtidos do IBGE
 Merge concluído: 5491/5569 municípios com código
  78 municípios sem código (verificar depois)


In [9]:
if df_atlas is not None:
    print("\n" + "="*60)
    print("📋 PREVIEW - ATLAS BRASIL LIMPO")
    print("="*60)
    
    print(f"\n📊 Shape final: {df_atlas_clean.shape}")
    print(f"\n📋 Primeiras 5 linhas:")
    display(df_atlas_clean.head())
    
    print(f"\n📊 Informações:")
    df_atlas_clean.info()


📋 PREVIEW - ATLAS BRASIL LIMPO

📊 Shape final: (5569, 24)

📋 Primeiras 5 linhas:


,territorialidades,razão_10perc_mais_ricos_40perc_mais_pobres,índice_de_gini,perc_15_a_24a_que_não_estudam_nem_trabalham_em_domicílios_vulneráveis_à_pobreza,perc_pessoas_em_domicílios_sem_energia_elétrica,perc_pessoas_em_domicílios_com_abastecimento_de_água_e_esgotamento_sanitário_inadequados,população_total,mortalidade_infantil,razão_de_dependência,taxa_de_envelhecimento,...,perc_25a+_com_ensino_superior_completo,expectativa_dea_de_estudo_aos_18a,renda_per_capita,perc_extremamente_pobres,perc_pobres,perc_vulneráveis_à_pobreza,municipio_uf,cod_municipio,municipio,uf
0,Brasil,22.78,0.60,11.61,1.42,6.12,190755799.0,16.7,45.87,7.36,...,11.27,9.54,793.87,6.62,15.20,32.56,NaN,NaN,NaN,NaN
1,Abadia de Goiás (GO),8.36,0.42,5.77,0.67,0.46,6876.0,13.4,44.64,5.73,...,7.23,7.94,574.96,1.97,6.18,23.27,Abadia de Goiás (GO),520005,Abadia de Goiás,GO
2,Abadia dos Dourados (MG),10.94,0.47,7.05,1.45,1.26,6704.0,14.8,42.76,9.77,...,5.96,8.68,596.18,1.85,7.94,27.40,Abadia dos Dourados (MG),310010,Abadia dos Dourados,MG
3,Abadiânia (GO),8.77,0.43,11.04,0.32,3.82,15757.0,12.6,44.72,7.13,...,4.92,9.31,519.87,2.00,8.45,31.19,Abadiânia (GO),520010,Abadiânia,GO
4,Abaeté (MG),14.85,0.54,9.27,0.08,0.17,22690.0,14.0,45.23,10.13,...,6.53,9.19,707.24,1.61,6.69,26.65,Abaeté (MG),310020,Abaeté,MG



📊 Informações:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5569 entries, 0 to 5568
Data columns (total 24 columns):
 #   Column                                                                                    Non-Null Count  Dtype  
---  ------                                                                                    --------------  -----  
 0   territorialidades                                                                         5569 non-null   object 
 1   razão_10perc_mais_ricos_40perc_mais_pobres                                                5566 non-null   object 
 2   índice_de_gini                                                                            5566 non-null   float64
 3   perc_15_a_24a_que_não_estudam_nem_trabalham_em_domicílios_vulneráveis_à_pobreza           5549 non-null   float64
 4   perc_pessoas_em_domicílios_sem_energia_elétrica                                           4875 non-null   float64
 5   perc_pessoas_em_domicílios_com_abasteci

## Salvar Atlas em parquet 

motivo: Trabalhar com parquet é uma estrutura mais leve, com isso eu tenho menor tempor de processamento. 


A função atlas_clean é porque na coluna "razão_10perc_mais_ricos_40perc_mais_pobres" o pandas/pyarrow esperava números (float ou int), mas encontrou strings e algumas em branco. Com isso vou converter para um tipo numérico.

In [ ]:
# Corrigir coluna problemática
df_atlas_clean['razão_10perc_mais_ricos_40perc_mais_pobres'] = (
    pd.to_numeric(
        df_atlas_clean['razão_10perc_mais_ricos_40perc_mais_pobres'].replace(' ', None),
        errors='coerce'
    )
)


if df_atlas is not None:
    print("\n" + "="*60)
    print(" SALVANDO ATLAS BRASIL")
    print("="*60)
    
    # Salvar em Parquet
    atlas_parquet = DATA_PROCESSED / "atlas_brasil_2010.parquet"
    df_atlas_clean.to_parquet(atlas_parquet, index=False)
    
    size_mb = os.path.getsize(atlas_parquet) / (1024**2)
    print(f" Salvo em Parquet: {atlas_parquet}")
    print(f" Tamanho: {size_mb:.2f} MB")


 SALVANDO ATLAS BRASIL
 Salvo em Parquet: ../data/processed/atlas_brasil_2010.parquet
 Tamanho: 0.57 MB


## Carregar Atlas no dunckDB

In [12]:
if df_atlas is not None:
    print("\n" + "="*60)
    print(" CARREGANDO ATLAS NO DUCKDB")
    print("="*60)
    
    atlas_parquet = DATA_PROCESSED / "atlas_brasil_2010.parquet"
    
    conn.execute(f"""
        CREATE OR REPLACE TABLE atlas_brasil AS
        SELECT * FROM '{atlas_parquet}'
    """)
    
    count = conn.execute("SELECT COUNT(*) FROM atlas_brasil").fetchone()[0]
    print(f" Tabela 'atlas_brasil' criada!")
    print(f" Registros: {count:,}")
    
    # Preview no DuckDB
    print(f"\n Preview (SQL):")
    preview = conn.execute("""
        SELECT 
            cod_municipio,
            municipio,
            uf,
            "perc_18a+_com_ensino_fundamental_completo",
            renda_per_capita,
            perc_pobres
        FROM atlas_brasil
        LIMIT 5
    """).df()
    display(preview)


 CARREGANDO ATLAS NO DUCKDB
 Tabela 'atlas_brasil' criada!
 Registros: 5,569

 Preview (SQL):


,cod_municipio,municipio,uf,perc_18a+_com_ensino_fundamental_completo,renda_per_capita,perc_pobres
0,None,None,None,54.92,793.87,15.20
1,520005,Abadia de Goiás,GO,48.86,574.96,6.18
2,310010,Abadia dos Dourados,MG,39.36,596.18,7.94
3,520010,Abadiânia,GO,43.27,519.87,8.45
4,310020,Abaeté,MG,36.32,707.24,6.69


## Carregar dados do SIM(Sistema de info. sobre mortaliadade) 
-> extraido do Tabnet

In [8]:
sim_files = sorted(DATA_RAW_SIM.glob("*.csv"))

if len(sim_files) > 0:
    print(f"\n Encontrados {len(sim_files)} arquivos")
    
    all_dfs = []
    
    for file in sim_files:
        print(f"    {file.name}...", end=" ")
        
        try:
            # Extrair ano do nome do arquivo
            # Formato: sim_cnv_obt-2014.csv
            year = file.stem.split('-')[-1]
            
            # Carregar CSV
            df_year = pd.read_csv(file, encoding='latin1', sep=';', skiprows=4)
            
            # Adicionar coluna de ano
            df_year['ano'] = int(year)
            
            all_dfs.append(df_year)
            print(f" {len(df_year):,} registros")
            
        except Exception as e:
            print(f" Erro: {e}")
    
    if len(all_dfs) > 0:
        # Concatenar todos os anos
        df_sim = pd.concat(all_dfs, ignore_index=True)
        
        print(f"\n Dados concatenados!")
        print(f" Total de registros: {len(df_sim):,}")
        print(f" Anos: {df_sim['ano'].min()} - {df_sim['ano'].max()}")
    else:
        print("\n Nenhum arquivo foi carregado!")
        df_sim = None
else:
    print("\n Nenhum arquivo SIM encontrado!")
    df_sim = None


 Encontrados 11 arquivos
    sim_cnv_obt-2014.csv...  36 registros
    sim_cnv_obt-2015.csv...  36 registros
    sim_cnv_obt-2016.csv...  36 registros
    sim_cnv_obt-2017.csv...  36 registros
    sim_cnv_obt-2018.csv...  36 registros
    sim_cnv_obt-2019.csv...  36 registros
    sim_cnv_obt-2020.csv...  36 registros
    sim_cnv_obt-2021.csv...  36 registros
    sim_cnv_obt-2022.csv...  36 registros
    sim_cnv_obt-2023.csv...  36 registros
    sim_cnv_obt-2024.csv...  36 registros

 Dados concatenados!
 Total de registros: 396
 Anos: 2014 - 2024


## Limpeza 

In [10]:
if df_sim is not None:
    print("\n" + "="*60)
    print("🧹 LIMPANDO DADOS DO SIM")
    print("="*60)

    print(f"\n Colunas originais:")
    print(df_sim.columns.tolist())

    # Renomear colunas de forma mais robusta
    col_rename = {}
    for col in df_sim.columns:
        col_lower = col.lower()
        if 'unidade da federação' in col_lower or col_lower == 'uf':
            col_rename[col] = 'uf'
        elif 'óbitos' in col_lower or 'obitos' in col_lower:
            col_rename[col] = 'total_obitos'

    df_sim_clean = df_sim.rename(columns=col_rename)

    print(f"\n✅ Colunas renomeadas:")
    for old, new in col_rename.items():
        print(f"   {old} → {new}")

    # Remover linhas com UF nula ou contendo "Total"
    if 'uf' in df_sim_clean.columns:
        df_sim_clean = df_sim_clean[
            df_sim_clean['uf'].notna() &
            ~df_sim_clean['uf'].str.contains('Total', case=False, na=False)
        ]

    # Converter 'total_obitos' para número (se existir)
    if 'total_obitos' in df_sim_clean.columns:
        df_sim_clean['total_obitos'] = (
            df_sim_clean['total_obitos']
            .astype(str)
            .str.replace('.', '', regex=False)
            .str.replace(',', '.', regex=False)
        )

        # Converte somente se possível
        df_sim_clean['total_obitos'] = pd.to_numeric(df_sim_clean['total_obitos'], errors='coerce')

    # Remover linhas onde total_obitos é nulo ou zero
    if 'total_obitos' in df_sim_clean.columns:
        df_sim_clean = df_sim_clean[df_sim_clean['total_obitos'].notna()]
        df_sim_clean = df_sim_clean[df_sim_clean['total_obitos'] > 0]

    print(f"\n Dados após limpeza: {len(df_sim_clean):,} registros")

    # Visualizar algumas linhas
    display(df_sim_clean.head(20))



🧹 LIMPANDO DADOS DO SIM

 Colunas originais:
['Unidade da Federação', 'Óbitos_p/Residênc', 'ano']

✅ Colunas renomeadas:
   Unidade da Federação → uf
   Óbitos_p/Residênc → total_obitos

 Dados após limpeza: 297 registros


,uf,total_obitos,ano
0,11 Rondônia,5740.0,2014
1,12 Acre,1770.0,2014
2,13 Amazonas,6940.0,2014
3,14 Roraima,1060.0,2014
4,15 Pará,21510.0,2014
5,16 Amapá,1460.0,2014
6,17 Tocantins,4960.0,2014
7,21 Maranhão,29690.0,2014
8,22 Piauí,16340.0,2014
9,23 Ceará,35450.0,2014


## Salvar SIM em parquet

In [13]:
if df_sim is not None:
    sim_parquet = DATA_PROCESSED / "sim_tabnet_2010_2024.parquet"
    df_sim_clean.to_parquet(sim_parquet, index=False)
        
    size_mb = os.path.getsize(sim_parquet) / (1024**2)
    print(f" Salvo em: {sim_parquet}")
    print(f" Tamanho: {size_mb:.2f} MB")

 Salvo em: ../data/processed/sim_tabnet_2010_2024.parquet
 Tamanho: 0.00 MB


## carregar SIM no dunckDB 

In [14]:
if df_sim is not None:
    
    sim_parquet = DATA_PROCESSED / "sim_tabnet_2010_2024.parquet"
    
    conn.execute(f"""
        CREATE OR REPLACE TABLE sim_tabnet AS
        SELECT * FROM '{sim_parquet}'
    """)
    
    count = conn.execute("SELECT COUNT(*) FROM sim_tabnet").fetchone()[0]
    print(f" Tabela 'sim_tabnet' criada!")
    print(f" Registros: {count:,}")


 Tabela 'sim_tabnet' criada!
 Registros: 297


## Visualizando as tabelas criadas no DunckDB 

In [15]:
tables = conn.execute("""
    SELECT table_name
    FROM information_schema.tables
    WHERE table_schema = 'main'
    ORDER BY table_name
""").fetchall()

print(f"\nTotal: {len(tables)} tabela(s)\n")

for table in tables:
    table_name = table[0]
    count = conn.execute(f"SELECT COUNT(*) FROM {table_name}").fetchone()[0]
    
    # Pegar colunas
    cols = conn.execute(f"SELECT COUNT(*) FROM information_schema.columns WHERE table_name = '{table_name}'").fetchone()[0]
    
    print(f" {table_name:20s} → {count:6,} linhas, {cols:2d} colunas")


Total: 2 tabela(s)

 atlas_brasil         →  5,569 linhas, 24 colunas
 sim_tabnet           →    297 linhas,  3 colunas


## Teste de querys no dunckdb 

In [ ]:
#teste 1: Ver municípios no Atlas
print("\nTop 10 municipios por IDHM:")
try:
    result = conn.execute("""
        SELECT 
            municipio,
            uf,
            ROUND(CAST(renda_per_capita AS FLOAT), 2) as renda_per_capita,
            ROUND(CAST(perc_pobres AS FLOAT), 2) as perc_pobres
        FROM atlas_brasil
        WHERE cod_municipio IS NOT NULL
        ORDER BY renda_per_capita DESC
        LIMIT 10
    """).df()
    display(result)
except Exception as e:
    print(f"    Erro: {e}")

# Teste 2: Ver dados do SIM por ano
print("\nObitos por ano (SIM TabNet):")
try:
    result = conn.execute("""
        SELECT 
            ano,
            COUNT(*) as num_registros,
            SUM(CAST(total_obitos AS INTEGER)) as total_obitos
        FROM sim_tabnet
        WHERE total_obitos IS NOT NULL
        GROUP BY ano
        ORDER BY ano
    """).df()
    display(result)
except Exception as e:
    print(f"    Erro: {e}")